# **EDA (Exploratory Data Analysis) - PRELIMINAR**

Durante el EDA, nos sumergimos en un conjunto de datos para comprender su estructura, abordar posibles outliers, identificar duplicados y gestionar valores faltantes. La riqueza de este análisis radica en la capacidad de revelar patrones, tendencias y relaciones entre variables, todo respaldado por visualizaciones pertinentes. A través de este proceso, buscamos extraer información valiosa que impulse futuras investigaciones y acciones relacionadas que nuestro cliente nos solicite.

In [1]:
#Se conecta Google Colaboratory con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


__IMPORTAMOS LIBRERIAS NECESARIAS__

In [2]:
#Instala pyspark en Google Colaboratory
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=57becd3c1e859f9bfcd5f1b6322eb2c8a6e5001589ea929cf0e4144ab43e4b16
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
#Se importan las librerias
import pandas as pd
import numpy as np
import ast
import json
import pickle
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, explode, sum,expr
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [4]:

spark = SparkSession.builder.appName("LecturaDeArchivos").getOrCreate()

__RUTA DE LOS ARCHIVOS Y CREACIÓN DEL DATAFRAME__

In [5]:
#Se cargan los archivos de la carpeta metadata-sitios y se compilan en un solo DataFrame

metadata_sitios=[]

for i in range(1,12):


    archivo = spark.read.json(f"/content/drive/MyDrive/Colab-Notebooks/metadata-sitios/{i}.json")

    archivo = archivo.withColumn("MISC", col("MISC").cast("string"))

    metadata_sitios.append(archivo)

df_final = metadata_sitios[0]

for dataframe in metadata_sitios[1:]:

  df_final = df_final.unionByName(dataframe)


metadata_sitios=df_final

In [6]:
metadata_sitios.show()

+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+--------------+-----+--------------------+--------------------+--------------------+
|                MISC|             address|avg_rating|            category|         description|             gmap_id|               hours|          latitude|          longitude|                name|num_of_reviews|price|    relative_results|               state|                 url|
+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+--------------+-----+--------------------+--------------------+--------------------+
|{[Wheelchair acce...|Porter Pharmacy, ...|       4.9|          [Pharmacy]|                NULL|0x88f16e41928ff68...|[[Friday, 8AM–6PM...|           32

Comenzamos a analizar este DataFrame sobre los diferentes negocios en los Estados Unidos, entre los datos podemos encontrar diferentes categorías como farmacias, restaurantes, cafeterias, gimnasios,... etc. Para nuestro proyecto es importante enfocarnos en el sector gastronómico, sin embargo en este Análisis de datos preliminar abarcaré el estudio de la calidad de los datos en general. En primera instancia, no tenemos una columna que indique explícitamente el Estado en el que se encuentra el negocio, sin embargo a futuro podemos construir una función que nos proporcione ese dato a partir de las coordenadas.

Comenzamos contando el número de registros de nuestro dataset para tener una dimensión del conjunto de datos que manejamos: 3025011. Ahora es importante conocer qué cantidad de datos faltan en cada columna para saber si es proporcionalmente admisible.

In [7]:
#Muestra el número de filas del DataFrame
metadata_sitios.count()

3025011

In [8]:
# Cuenta el número de nulos en cada columna

def conteo_nulos(dataframe):
  conteo_nulos_por_columna = dataframe.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in dataframe.columns])

  # Muestra el resultado
  conteo_nulos_por_columna.show()


In [9]:
conteo_nulos(metadata_sitios)

+------+-------+----------+--------+-----------+-------+------+--------+---------+----+--------------+-------+----------------+------+---+
|  MISC|address|avg_rating|category|description|gmap_id| hours|latitude|longitude|name|num_of_reviews|  price|relative_results| state|url|
+------+-------+----------+--------+-----------+-------+------+--------+---------+----+--------------+-------+----------------+------+---+
|690834|  80511|         0|   17419|    2770722|      0|787405|       0|        0|  37|             0|2749808|          295058|746455|  0|
+------+-------+----------+--------+-----------+-------+------+--------+---------+----+--------------+-------+----------------+------+---+



Como se puede observar los campos 'description' y 'price' tienen demasiados datos nulos en proporción al total de registros como para ser siquiera considerados. También otras columnas como 'state' y 'MISC' no son de relevancia para nuestro estudio, pero esto se analizará más adelante cuando tengamos que hacer el ETL y normalizar los Datasets.

Ahora nos interesa conocer las categorías de negocio en específico que contienen nuestros datos.

In [10]:
# Explota la columna 'category' y obtiene elementos únicos
categorias_sin_duplicados = metadata_sitios.select(explode("category").alias("categoria")).distinct()

# Colecta los resultados en una lista
lista_categorias_sin_duplicados = [row.categoria for row in categorias_sin_duplicados.collect()]

# Muestra la lista resultante
print("Categorías sin duplicados:", lista_categorias_sin_duplicados)

# Muestra cuantas categorías hay en total

print("El número de categorías es: ", len(lista_categorias_sin_duplicados))

#Ahora voy a contar cuantas de esas categorías son de restaurantes

contador=0

for elemento in lista_categorias_sin_duplicados:

  if 'restaurant' in elemento:

    contador+=1

print(f"Los restaurantes abarcan el {round(contador*100/len(lista_categorias_sin_duplicados),2)}% de las categorías")

Categorías sin duplicados: ['Art gallery', 'Dry wall contractor', 'Sports massage therapist', 'Tile store', 'Basketball club', 'Leather coats store', 'Public relations firm', 'Central American restaurant', 'Taxicab Stand', 'Wedding dress rental service', 'Data entry service', 'Debt collecting', 'Law book store', 'Gardening products and services', 'Deck builder', 'Professional organizer', 'Textile mill', 'Racquetball club', 'RV dealer', 'Water works equipment supplier', 'Mental health', 'Curtain supplier and maker', 'Timeshare agency', 'Dog cafe', 'Pony ride service', 'Fruits wholesaler', 'Gasket manufacturer', 'Archaeological site', 'Mobile home park', 'Emergency room', 'Landscape lighting designer', 'Plastic surgeon', 'Gay bar', 'Data recovery service', 'Wedding bakery', 'Market', 'Salvage dealer', 'Services', 'Mercedes-Benz dealer', 'Mens tailor', 'Orthodox church', 'Computer accessories store', 'Educational testing service', 'Haunted house', 'Haute couture fashion house', 'Olive oil

Es de interés conocer cuantos de los registros tienen la cadena 'restaurant' en alguna de sus categorías, esto es lo mismo que responder a la pregunta: ¿Cuántos de estos negocios son restaurantes?.

In [11]:
df_aux = metadata_sitios.withColumn("category", col("category").cast("string"))

# Filtra y cuenta las filas que contienen 'restaurant' en la columna 'category'
contador_restaurant = df_aux.filter(col("category").contains('restaurant')).count()


# Imprime el resultado
print("El número de restaurantes es:", contador_restaurant)

El número de restaurantes es: 152198


Esto quiere decir que aproximadamente el 5% de los registros del Dataset "metadata_sitios" corresponde a restaurantes, conocer esto es de bastante utilidad para dimensionar el tamaño de los archivos que vamos a normalizar y cargar en nuestra base de datos.

In [12]:
df_aux = metadata_sitios.withColumn("category", col("category").cast("string"))

# Filtra y cuenta las filas que contienen 'Vegetarian restaurant' en la columna 'category'
contador_fastfood = df_aux.filter(col("category").contains('Pizza restaurant')).count()


# Imprime el resultado
print("El número de restaurantes de pizzeria es:", contador_fastfood)
print(f"Representa el: {round(contador_fastfood*100/contador_restaurant,2)}% de los restaurantes")

El número de restaurantes de pizzeria es: 23814
Representa el: 15.65% de los restaurantes


Proseguiremos a explorar los demás DataFrames, analizando los datos nulos de manera similar.

In [13]:
#Se leen los reviews de Nevada

reviews_NewYork=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/MyDrive/Colab-Notebooks/reviews-estados/reviews-New_York/{i}.json")



        reviews_NewYork.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_NewYork[0]

for dataframe in reviews_NewYork[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_NewYork=df_final


In [14]:
reviews_NewYork.show()

+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+--------------------+
|             gmap_id|                name|                pics|rating|                resp|                text|         time|             user_id|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+--------------------+
|0x89c25fc9494dce4...|      Alvin Martinez|[{[https://lh5.go...|     5|                NULL|I'm late to posti...|1603494795361|11372210469230823...|
|0x89c25fc9494dce4...|     Johnnie Jackson|                NULL|     1|{We pride ourselv...|Very dissatisfied...|1620157037403|10729344149210932...|
|0x89c25fc9494dce4...|        Manie Blazer|                NULL|     5|                NULL|Excellent very we...|1597431662039|10037858580181940...|
|0x89c25fc9494dce4...|      Fashion Fiinds|                NULL|     5|{Thanks for the a...|Basing my revi

In [15]:
#Muestra el número de filas del DataFrame
reviews_NewYork.count()


150000

In [16]:
conteo_nulos(reviews_NewYork)

+-------+----+------+------+------+-----+----+-------+
|gmap_id|name|  pics|rating|  resp| text|time|user_id|
+-------+----+------+------+------+-----+----+-------+
|      0|   0|145404|     0|133162|62676|   0|      0|
+-------+----+------+------+------+-----+----+-------+



En este caso las columnas 'pics' y 'resp' tienen muchos datos nulos como para ser consideradas, de manera similar ocurre con los otros dos Estados. También nos damos cuenta que la columna 'gmap_id' jugará un papel clave para identificar cada negocio en el Dataset de metadata_sitios

In [17]:
#Se leen los reviews de Florida

reviews_California=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/MyDrive/Colab-Notebooks/reviews-estados/reviews-California/{i}.json")



        reviews_California.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_California[0]

for dataframe in reviews_California[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_California=df_final

In [18]:
reviews_California.show()

+--------------------+-------------+----+------+----+--------------------+-------------+--------------------+
|             gmap_id|         name|pics|rating|resp|                text|         time|             user_id|
+--------------------+-------------+----+------+----+--------------------+-------------+--------------------+
|0x80c2c778e3b73d3...|      Song Ro|NULL|     5|NULL|Love there korean...|1609909927056|10899115226265578...|
|0x80c2c778e3b73d3...|  Rafa Robles|NULL|     5|NULL|      Good very good|1612849648663|11129032221979621...|
|0x80c2c778e3b73d3...|    David Han|NULL|     4|NULL|They make Korean ...|1583643882296|11264035744961195...|
|0x80c2c778e3b73d3...|  Anthony Kim|NULL|     5|NULL|Short ribs are ve...|1551938216355|11744034972382365...|
|0x80c2c778e3b73d3...|Mario Marzouk|NULL|     5|NULL|Great food and pr...|1494910901933|10058077083612353...|
|0x80c2c778e3b73d3...|  Ana Salazar|NULL|     5|NULL|This food is deli...|1547799582640|10018573266445490...|
|0x80c2c77

In [19]:
reviews_California.count()

2100000

In [20]:
conteo_nulos(reviews_California)

+-------+----+-------+------+-------+------+----+-------+
|gmap_id|name|   pics|rating|   resp|  text|time|user_id|
+-------+----+-------+------+-------+------+----+-------+
|      0|   0|2022925|     0|1896377|895276|   0|      0|
+-------+----+-------+------+-------+------+----+-------+



### DATASET: __YELP__

In [21]:
#Se lee el archivo business de yelp

with open("/content/drive/MyDrive/Colab-Notebooks/Yelp/business.pkl", 'rb') as archivo:

        business_yelp = pickle.load(archivo)


In [22]:
business_yelp.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
business_yelp.shape[0]

150346

In [24]:

def contar_nulos(dataframe):

    # Obtener la cantidad de valores nulos por columna
  nulos_por_columna = dataframe.isnull().sum()

  print("Cantidad de valores nulos por columna:\n", nulos_por_columna)



In [25]:
contar_nulos(business_yelp)

Cantidad de valores nulos por columna:
 business_id          0
name                 0
address              0
city                 0
state                3
postal_code          0
latitude             0
longitude            0
stars                0
review_count         0
is_open              0
attributes       13744
categories         103
hours            23223
business_id     150341
name            150341
address         150341
city            150341
state           150341
postal_code     150341
latitude        150341
longitude       150341
stars           150341
review_count    150341
is_open         150341
attributes      150341
categories      150341
hours           150341
dtype: int64


In [26]:
#Elimina las columnas repetidas que tienen casi todos sus datos nulos
business_yelp=business_yelp.iloc[:,0:14]

Este Dataset 'business yelp' tiene muchas columnas repetidas, en su mayoría los datos de esas copias son nulos, para nuestro trabajo deben ser eliminadas. Es importante conocer el número de negocios que pertenecen a los Estados que son objeto de nuestro estudio

In [27]:
business_yelp[(business_yelp['state']=='NV') | ((business_yelp['state']=='FL') | (business_yelp['state']=='VA'))].shape[0]

34044

In [28]:
proporcion= round(34044*100/150346,2)

print(f"El porcentaje de datos que corresponde a alguno de los tres Estados es: {proporcion}%")

El porcentaje de datos que corresponde a alguno de los tres Estados es: 22.64%


In [29]:
contador= 0
for elemento in business_yelp['categories']:

  try:

    if 'Restaurants' in elemento:

      contador+=1

  except:

    continue

print("La cantidad total de restaurantes es: ", contador)
print(f"Representan un {round(contador*100/150346,2)}% de los negocios")

La cantidad total de restaurantes es:  52268
Representan un 34.77% de los negocios


In [30]:
#Se lee el archivo checkin de yelp

checkin_yelp=[]

with open("//content/drive/MyDrive/Colab-Notebooks/Yelp/checkin.json","r", encoding= 'utf-8') as filejson:


                # Procesa cada línea del archivo como un objeto JSON
                for linea in filejson:

                        # Intenta cargar la línea como un objeto JSON
                        objeto = json.loads(linea)

                        checkin_yelp.append(objeto)

checkin_yelp=pd.DataFrame(checkin_yelp)



In [31]:
checkin_yelp.head(5)


,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [32]:

checkin_yelp.shape[0]

131930

In [33]:

contar_nulos(checkin_yelp)

Cantidad de valores nulos por columna:
 business_id    0
date           0
dtype: int64


In [ ]:
#Se lee el archivo review de yelp

review_yelp = spark.read.json("/content/drive/My Drive/Yelp/review.json")

In [ ]:
review_yelp.show()

In [ ]:
review_yelp.count()

In [ ]:

conteo_nulos(review_yelp)

En este caso no hay valores nulos, lo que es bastante positivo para nuestro sistema de recomendación y para facilitar la limpieza de datos.

In [ ]:
#Se lee el archivo tip de yelp

tip_yelp=[]

with open("/content/drive/My Drive/Yelp/tip.json","r", encoding= 'utf-8') as filejson:


                # Procesa cada línea del archivo como un objeto JSON
                for linea in filejson:

                        # Intenta cargar la línea como un objeto JSON
                        objeto = json.loads(linea)

                        tip_yelp.append(objeto)

tip_yelp=pd.DataFrame(tip_yelp)

In [ ]:

tip_yelp.head(5)

In [ ]:

tip_yelp.shape[0]


In [ ]:
contar_nulos(tip_yelp)

In [ ]:
#Se lee el archivo user de yelp

user_yelp= pd.read_parquet("/content/drive/My Drive/Yelp/user.parquet")


In [ ]:

user_yelp.head(5)

In [ ]:
user_yelp.shape[0]

In [ ]:
contar_nulos(user_yelp)


Como se pudo observar, los datos de Yelp no necesitan mayor tratamiento en cuanto a datos faltantes.